In [ ]:
## Create the clean files where 1 visit patients are removed

import pandas as pd

cvFolds = 10

for i in range(cvFolds):
    print("Fold Number {}:\n".format(i))
    filename = '/PatientTM/data/extended_folds/discharge_subjectsplit/fold' + str(i)
    extra = '_clean'
    extension = '_notext.csv'
    foldDf = pd.read_csv(filename+extension).drop(["Unnamed: 0"], axis=1)
    foldDfHadmCount = foldDf.groupby(['SUBJECT_ID'], as_index=False).agg(numAdmissions=('SUBJECT_ID', lambda x: len(x)))
    validSubjectIds = foldDfHadmCount[foldDfHadmCount.numAdmissions > 1].SUBJECT_ID
    cleanFoldDf = foldDf[foldDf.SUBJECT_ID.isin(validSubjectIds)].reset_index(drop=True)
    # print(cleanFoldDf.sort_values(['SUBJECT_ID','ADMITTIME'], ascending=True)) 
    # print(cleanFoldDf['Label'].value_counts())
    cleanFoldDf.to_csv(filename+extra+extension)

In [2]:
# READMISSION - Order admission by time and remove the last admission
import pandas as pd

cvFolds = 10

for i in range(cvFolds):
    print("Fold Number {}:\n".format(i))
    filename1 = '/PatientTM/data/extended_folds/discharge_subjectsplit/fold' + str(i)
    extension1 = '_clean_notext.csv'
    filename2 = '/PatientTM/data/extended_folds/discharge_subjectsplit_trajectory/fold' + str(i)
    extension2 = '_allvisits_readmission_notext.csv'
   
    foldDf = pd.read_csv(filename1+extension1).drop(["Unnamed: 0"], axis=1)
    foldDf = foldDf.sort_values(['SUBJECT_ID','ADMITTIME'], ascending=True)
    lastVisits = foldDf.groupby(['SUBJECT_ID'], as_index=False).nth(-1)
    # Taking the last visit from the dataframe as we cannot use them for prediction here
    remainingVisitsDf = foldDf[~foldDf.HADM_ID.isin(lastVisits.HADM_ID)]
    remainingVisitsDf.to_csv(filename2+extension2)
    
    # numAdmissions = remainingVisitsDf.groupby(['SUBJECT_ID'], as_index=False).agg(numAdmissions=('SUBJECT_ID', lambda x: len(x)))
    # print(max(numAdmissions.numAdmissions))
    # print(numAdmissions[numAdmissions.numAdmissions.isin(["8.0"])].SUBJECT_ID)
    # print(last6Visits['Label'].value_counts())

Fold Number 0:

Fold Number 1:

Fold Number 2:

Fold Number 3:

Fold Number 4:

Fold Number 5:

Fold Number 6:

Fold Number 7:

Fold Number 8:

Fold Number 9:



In [15]:
# READMISSION NEW VERSION, SLIDING WINDOW - Structure data in a new matrix file where data is structured by patient and has the visit history organized as well as the label

cvFolds = 10
numberOfHistoryAdmissions = 6

for i in range(cvFolds):
    patientMatrix = []
    print("Fold Number {}:\n".format(i))
    filename1 = '/PatientTM/data/extended_folds/discharge_subjectsplit_trajectory/fold' + str(i)
    extension1 = '_allvisits_readmission_notext.csv'
    filename2 = '/PatientTM/data/extended_folds/discharge_subjectsplit_trajectory/fold' + str(i)
    extension2 = '_sliding'+str(numberOfHistoryAdmissions)+'trajectory_readmission_notext.csv'
    foldDf = pd.read_csv(filename1+extension1).drop(["Unnamed: 0"], axis=1) 
    patientVisits = []
    patientVisitsWindow = []
    prevSubject = ""
    prevLabel = ""
    
    for index in foldDf.index:
        subject_id = foldDf.loc[index, "SUBJECT_ID"]        
        if prevSubject != subject_id:
            while len(patientVisitsWindow) < numberOfHistoryAdmissions:
                patientVisitsWindow.append(0.0)
            if len(patientVisitsWindow) == numberOfHistoryAdmissions:
                patientVisits.append(prevSubject)
                for visit in patientVisitsWindow:
                    patientVisits.append(visit)
                patientVisits.append(prevLabel)
            patientMatrix.append(patientVisits)
            patientVisits, patientVisitsWindow = [], []
            
            prevSubject = subject_id
            prevLabel = foldDf.loc[index, "Label"]
            patientVisitsWindow.append(foldDf.loc[index, "HADM_ID"])
        elif prevSubject == subject_id:
            if len(patientVisitsWindow) == numberOfHistoryAdmissions:
                patientVisits.append(prevSubject)
                for visit in patientVisitsWindow:
                    patientVisits.append(visit)
                patientVisits.append(prevLabel)
                patientMatrix.append(patientVisits)
                
                patientVisits = []
                del patientVisitsWindow[0] #Remove the first entry to open space for a new more recent visit
                
            prevLabel = foldDf.loc[index, "Label"]
            patientVisitsWindow.append(foldDf.loc[index, "HADM_ID"])
                
    #Process and append last patient
    while len(patientVisitsWindow) < numberOfHistoryAdmissions:
        patientVisitsWindow.append(0.0)
    patientVisits.append(prevSubject)
    for visit in patientVisitsWindow:
        patientVisits.append(visit)
    patientVisits.append(prevLabel)
    patientMatrix.append(patientVisits)
    #Remove first dummy entry
    patientMatrix = patientMatrix[1:]
    columnHeaders = ['SUBJECT_ID']
    for i in range(numberOfHistoryAdmissions):
        columnHeaders.append('VISIT_'+str(i+1))
    columnHeaders.append('Label')
    patientMatrixDf = pd.DataFrame(patientMatrix, columns = columnHeaders)
    patientMatrixDf.to_csv(filename2+extension2)

Fold Number 0:

Fold Number 1:

Fold Number 2:

Fold Number 3:

Fold Number 4:

Fold Number 5:

Fold Number 6:

Fold Number 7:

Fold Number 8:

Fold Number 9:



In [3]:
# DIAGNOSIS - Order admission by time. The last visit was already removed priorly
cvFolds = 10

for i in range(cvFolds):
    print("Fold Number {}:\n".format(i))
    filename1 = '/PatientTM/data/extended_folds/discharge_subjectsplit/fold' + str(i)
    extension1 = '_codeprediction_notext.csv'
    filename2 = '/PatientTM/data/extended_folds/discharge_subjectsplit_trajectory/fold' + str(i)
    extension2 = '_allvisits_diagnosis_notext.csv'
   
    foldDf = pd.read_csv(filename1+extension1).drop(["Unnamed: 0"], axis=1)  
    foldDf = foldDf.sort_values(['SUBJECT_ID','ADMITTIME'], ascending=True)
    foldDf.to_csv(filename2+extension2)
    

Fold Number 0:

Fold Number 1:

Fold Number 2:

Fold Number 3:

Fold Number 4:

Fold Number 5:

Fold Number 6:

Fold Number 7:

Fold Number 8:

Fold Number 9:



In [18]:
# DIAGNOSES NEW VERSION, SLIDING WINDOW - Structure data in a new matrix file where data is structured by patient and has the visit history organized as well as the label

cvFolds = 10
numberOfHistoryAdmissions = 6

for i in range(cvFolds):
    patientMatrix = []
    print("Fold Number {}:\n".format(i))
    filename1 = '/PatientTM/data/extended_folds/discharge_subjectsplit_trajectory/fold' + str(i)
    extension1 = '_allvisits_diagnosis_notext.csv'
    filename2 = '/PatientTM/data/extended_folds/discharge_subjectsplit_trajectory/fold' + str(i)
    extension2 = '_sliding'+str(numberOfHistoryAdmissions)+'trajectory_diagnosis_notext.csv'
    foldDf = pd.read_csv(filename1+extension1).drop(["Unnamed: 0"], axis=1) 
    patientVisits = []
    patientVisitsWindow = []
    prevSubject = ""
    prevLabelIcd = ""
    prevLabelCcs = ""
    
    for index in foldDf.index:
        subject_id = foldDf.loc[index, "SUBJECT_ID"]        
        if prevSubject != subject_id:
            while len(patientVisitsWindow) < numberOfHistoryAdmissions:
                patientVisitsWindow.append(0.0)
            if len(patientVisitsWindow) == numberOfHistoryAdmissions:
                patientVisits.append(prevSubject)
                for visit in patientVisitsWindow:
                    patientVisits.append(visit)
                patientVisits.append(prevLabelIcd)
                patientVisits.append(prevLabelCcs)
            patientMatrix.append(patientVisits)
            patientVisits, patientVisitsWindow = [], []
            
            prevSubject = subject_id
            prevLabelIcd = foldDf.loc[index, "LABEL_NEXT_SMALL_DIAG_ICD9"]
            prevLabelCcs = foldDf.loc[index, "LABEL_NEXT_DIAG_CCS"]
            patientVisitsWindow.append(foldDf.loc[index, "HADM_ID"])
        elif prevSubject == subject_id:
            if len(patientVisitsWindow) == numberOfHistoryAdmissions:
                patientVisits.append(prevSubject)
                for visit in patientVisitsWindow:
                    patientVisits.append(visit)
                patientVisits.append(prevLabelIcd)
                patientVisits.append(prevLabelCcs)
                patientMatrix.append(patientVisits)
                
                patientVisits = []
                del patientVisitsWindow[0] #Remove the first entry to open space for a new more recent visit
                
            prevLabelIcd = foldDf.loc[index, "LABEL_NEXT_SMALL_DIAG_ICD9"]
            prevLabelCcs = foldDf.loc[index, "LABEL_NEXT_DIAG_CCS"]
            patientVisitsWindow.append(foldDf.loc[index, "HADM_ID"])
                
    #Process and append last patient
    while len(patientVisitsWindow) < numberOfHistoryAdmissions:
        patientVisitsWindow.append(0.0)
    patientVisits.append(prevSubject)
    for visit in patientVisitsWindow:
        patientVisits.append(visit)
    patientVisits.append(prevLabelIcd)
    patientVisits.append(prevLabelCcs)
    patientMatrix.append(patientVisits)
    #Remove first dummy entry
    patientMatrix = patientMatrix[1:]
    columnHeaders = ['SUBJECT_ID']
    for i in range(numberOfHistoryAdmissions):
        columnHeaders.append('VISIT_'+str(i+1))
    columnHeaders.append('LABEL_NEXT_SMALL_DIAG_ICD9')
    columnHeaders.append('LABEL_NEXT_DIAG_CCS')
    patientMatrixDf = pd.DataFrame(patientMatrix, columns = columnHeaders)
    patientMatrixDf.to_csv(filename2+extension2)

Fold Number 0:

Fold Number 1:

Fold Number 2:

Fold Number 3:

Fold Number 4:

Fold Number 5:

Fold Number 6:

Fold Number 7:

Fold Number 8:

Fold Number 9:



In [28]:
# CONVERT DIAG_CCS INPUT FEATURE TO MULTI-ONE-HOT

import json
import pandas as pd
import numpy as np

def processString(string, charsToRemove):
    for char in charsToRemove: string = string.replace(char, "")
    return string

with open("../data/extended/preprocessing/ICDandCCSmappings/merged_ccs_text.json") as file:
    ccsFile = json.load(file)
    
diagMap = {}
diagIdx = 0

ccsFile = {key: value for key, value in sorted(ccsFile.items(), key=lambda item: item[0])}

for value in ccsFile:
    if value.startswith("D_"):
        diagMap[value] = diagIdx
        diagIdx += 1

        
print(f"The resulting diagMap has {len(diagMap)} entries")
         
cvFolds = 10
numCCSCodes = len(diagMap)

for i in range(cvFolds):
    print("Fold Number {}:\n".format(i))
    origin_filename = "../data/extended_folds/discharge_subjectsplit_trajectory/fold"+str(i)+"_allvisits_diagnosis_notext_old.csv"
    destination_filename = "../data/extended_folds/discharge_subjectsplit_trajectory/fold"+str(i)+"_allvisits_diagnosis_notext.csv"
    foldDf = pd.read_csv(origin_filename).drop(["Unnamed: 0"], axis=1)
    
    #Adding new column for label
    foldDf['LABEL_DIAG_CCS'] = np.nan
    foldDf['LABEL_DIAG_CCS'] = foldDf['LABEL_DIAG_CCS'].astype(object)
    
    for index in foldDf.index:
        labels = np.zeros(numCCSCodes, dtype=np.int)
        ccsCodes = [x for x in processString(foldDf.loc[index, 'DIAG_CCS'], charsToRemove = "[]\' ").split(',')]
        for code in ccsCodes:
            # print(code, diagMap[code])
            labelIndex = diagMap[code]
            labels[labelIndex] = 1
        foldDf.at[index, 'LABEL_DIAG_CCS'] = list(labels)
    foldDf.to_csv(destination_filename)
    

The resulting diagMap has 255 entries
Fold Number 0:

Fold Number 1:

Fold Number 2:

Fold Number 3:

Fold Number 4:

Fold Number 5:

Fold Number 6:

Fold Number 7:

Fold Number 8:

Fold Number 9:

